Review 5

In [ ]:
using System;
using System.Collections.Generic;

// Перечисление для статусов заказов
public enum OrderStatus
{
    Created,
    InProgress,
    Completed
}

// Класс для представления заказчиков
public class Customer
{
    public string FirstName { get; set; } // Имя заказчика
    public string LastName { get; set; }  // Фамилия заказчика
    public string PhoneNumber { get; set; } // Контактный номер телефона

    public Customer(string firstName, string lastName, string phoneNumber)
    {
        FirstName = firstName;
        LastName = lastName;
        PhoneNumber = phoneNumber;
    }
}

// Класс для представления сотрудников компании
public class Employee
{
    public string FirstName { get; set; } // Имя сотрудника
    public string LastName { get; set; }  // Фамилия сотрудника
    public string Position { get; set; }   // Должность
    public int OrdersProcessed { get; set; } = 0; // Количество обработанных заказов

    public Employee(string firstName, string lastName, string position)
    {
        FirstName = firstName;
        LastName = lastName;
        Position = position;
    }
}

// Класс для представления заказов
public class Order
{
    public int OrderId { get; private set; } // Уникальный идентификатор заказа
    public string Description { get; set; } // Описание заказа
    public Employee AssignedEmployee { get; set; } // Ответственный сотрудник
    public OrderStatus Status { get; set; } // Статус заказа
    public DateTime CreatedAt { get; private set; } // Дата создания заказа

    // Статический идентификатор для генерации уникальных идентификаторов заказов
    private static int nextId = 1;

    public Order(string description)
    {
        OrderId = nextId++; // Автоматическая генерация идентификатора
        Description = description;
        Status = OrderStatus.Created; // Статус при создании заказа "Создан"
        CreatedAt = DateTime.Now; // Установка даты создания заказа
    }
}

// Класс для представления компании
public class Company
{
    public string Name { get; set; } // Название компании
    private List Employees { get; set; } = new List(); // Список сотрудников
    private List Orders { get; set; } = new List(); // Список заказов

    // Делегат для уведомления об изменении статуса заказа
    public delegate void OrderStatusChangedHandler(Order order);
    public event OrderStatusChangedHandler OrderStatusChanged; // Событие изменения статуса заказа

    public Company(string name)
    {
        Name = name;
    }

    // Метод для добавления нового сотрудника
    public void AddEmployee(Employee employee)
    {
        Employees.Add(employee);
        Console.WriteLine($"Сотрудник {employee.FirstName} {employee.LastName} добавлен в компанию.");
    }

    // Метод для добавления нового заказа
    public void AddOrder(Order order)
    {
        Orders.Add(order);
        Console.WriteLine($"Заказ {order.Description} добавлен. ID: {order.OrderId}");
    }

    // Метод для назначения сотрудника на заказ
    public void AssignEmployeeToOrder(int orderId, Employee employee)
    {
        var order = Orders.Find(o => o.OrderId == orderId); // Поиск заказа по идентификатору
        if (order != null)
        {
            order.AssignedEmployee = employee; // Назначение сотрудника
            Console.WriteLine($"Сотрудник {employee.FirstName} {employee.LastName} назначен на заказ ID: {order.OrderId}");
        }
        else
        {
            Console.WriteLine($"Заказ с ID {orderId} не найден.");
        }
    }

    // Метод для изменения статуса заказа
    public void ChangeOrderStatus(int orderId, OrderStatus newStatus)
    {
        var order = Orders.Find(o => o.OrderId == orderId); // Поиск заказа по идентификатору
        if (order != null)
        {
            order.Status = newStatus; // Изменение статуса заказа
            Console.WriteLine($"Статус заказа ID: {order.OrderId} изменен на {newStatus}.");
            
            // Вызываем событие для уведомления сотрудников
            OrderStatusChanged?.Invoke(order);
        }
        else
        {
            Console.WriteLine($"Заказ с ID {orderId} не найден.");
        }
    }

    // Метод для просмотра всех заказов по статусу
    public List GetOrdersByStatus(OrderStatus status)
    {
        return Orders.FindAll(o => o.Status == status); // Возвращаем список заказов по статусу
    }

    // Метод для получения отчета по всем сотрудникам
    public void PrintEmployeeReport()
    {
        Console.WriteLine("\nОтчет по сотрудникам:");
        foreach (var employee in Employees)
        {
            Console.WriteLine($"{employee.FirstName} {employee.LastName} - обработано заказов: {employee.OrdersProcessed}");
        }
    }
}


// Создание компании
Company company = new Company("Технологические Решения");

// Подписка на событие изменения статуса заказа
company.OrderStatusChanged += (Order order) =>
{
    Console.WriteLine($"Заказ ID: {order.OrderId} изменил статус на {order.Status}.");
    if (order.AssignedEmployee != null)
    {
        order.AssignedEmployee.OrdersProcessed++; // Увеличиваем количество обработанных заказов сотрудника
    }
};

// Добавление сотрудников
Employee alice = new Employee("Алиса", "Иванова", "Менеджер");
Employee bob = new Employee("Боб", "Смирнов", "Разработчик");
company.AddEmployee(alice);
company.AddEmployee(bob);

// Создание и добавление заказов
Order order1 = new Order("Разработка веб-сайта");
Order order2 = new Order("Создание мобильного приложения");
company.AddOrder(order1);
company.AddOrder(order2);

// Назначение сотрудников на заказы
company.AssignEmployeeToOrder(order1.OrderId, alice);
company.AssignEmployeeToOrder(order2.OrderId, bob);

// Изменение статуса заказа
company.ChangeOrderStatus(order1.OrderId, OrderStatus.InProgress);
company.ChangeOrderStatus(order2.OrderId, OrderStatus.Completed);

// Получение отчета по сотрудникам
company.PrintEmployeeReport();
